# TipTrack CNN Training

In [ ]:
from keras.models import Sequential
from keras.metrics import categorical_crossentropy
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, RandomFlip, RandomRotation
# from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomBrightness

from prepare_dataset import PrepareDataset

In [ ]:
IMG_SIZE = 48

TARGET_FOLDER = 'training_images/2023-02-22'

MODEL_NAME = 'model_2023_003'

In [ ]:
prepare_dataset = PrepareDataset()

train_X, train_label, test_X, test_label = prepare_dataset.get_dataset(TARGET_FOLDER, IMG_SIZE)

## Init Keras Model

In [ ]:
# batch 64
# epoch 1
# conv linear

batch_size = 128
epochs = 1
num_classes = 2
activation = 'relu'
activation_conv = 'linear'  # LeakyReLU
layer_count = 5
num_neurons = 128

# Define model structure
model = Sequential()

# model.add(data_augmentation_new)
model.add(
    Conv2D(64, kernel_size=(3, 1), activation=activation_conv, input_shape=(IMG_SIZE, IMG_SIZE, 1), padding='same'))
model.add(Conv2D(64, (1, 3), activation=activation_conv, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Conv2D(32, (3, 1), activation=activation_conv, padding='same'))
model.add(Conv2D(32, (1, 3), activation=activation_conv, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
# model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation=activation_conv, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(0.2))
# model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())

# trial and error: linear performs much better than ReLU and sigmoid
for i in range(layer_count - 1):
    model.add(Dense(num_neurons, activation=activation))
model.add(Dropout(0.2))

model.add(Dense(num_neurons, activation=activation))
# model.add(Dense(64, activation=activation))
# model.add(Dense(64, activation=activation))
# model.add(Dense(128, activation='linear'))
# model.add(Dense(128, activation='linear'))

# classifier
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=categorical_crossentropy, optimizer="adam", metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)

## Train Model

In [ ]:
history = model.fit(
    train_X,
    train_label,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(test_X, test_label),
    callbacks=[reduce_lr]
)

# TODO: Create a plot from these values
loss = print(history.history['loss'])
val_loss = print(history.history['val_loss'])
accuracy = print(history.history['accuracy'])
val_accuracy = print(history.history['val_accuracy'])

## Save Model

In [ ]:
if MODEL_NAME in next(os.walk('.'))[1]:
    print('Warning: There already exists a model called "{}"'.format(MODEL_NAME))
    print('Make sure you want to overwrite it before it gets saved!')
else:
    model.save(MODEL_NAME)

model.summary()